# Module 2: Preprocessing, indexing and approximate matching

## Boyer-Moore

* an efficient string-searching algorithm
* preprocesses the pattern **P** (the string being searched for), but not the text **T** (the string being searched in)
* well-suited for applications in which the pattern is much shorter than the text or where it persists across multiple searches
* uses information gathered during the preprocess step to skip sections of the text
* for each alignment the last character in P is compared first

### Bad character rule

* considers the character in T at which the comparison process failed (assuming such a failure occurred)
* upon missmatch, skip alignments until the missmatch becomes a match or pattern moves past missmatched character.

E.g. 

alignment 1: missmatch
```
T: GCTTCTGCTACCTTTT
P: CCTTTTGC
       x✓✓✓
```

alignment 2: still a missmatch - skip
```
T: GCTTCTGCTACCTTTT
P:  CCTTTTGC
       x
```

alignment 3: still a missmatch - skip
```
T: GCTTCTGCTACCTTTT
P:   CCTTTTGC
       x
```

alignment 4: match - do not skip
```
T: GCTTCTGCTACCTTTT
P:    CCTTTTGC
       ✓
```

### Good suffix rule

## K-mer indexing

* `k`-mer = substring of `t` of length `k`

* For every `k`-mer in `t`, record the index (aka the offset) where it occurs in `t`

* For a given pattern `p` can find all the offsets where `p` occurs in `t` using the `k`-mer index

  * e.g. take the first `k`-mer in `p` and look it up in the index

  * e.g. if a match is found at offset `i` then we know that `p[:k] == t[i:i+k]`
  
  * next need to verify that the rest of `p` also matches `t` - i.e. we need to check that `p[k:] == t[i+k:i+len(p)]`

### Variations

* Use every other k-mer 

## Approximate matching

Hamming distance

Edit distance (aka Levenstein distance)



## Pigeonhole principle